
<h1 id="Adversarial-Machine-Learning">Adversarial Machine Learning<a class="anchor-link" href="#Adversarial-Machine-Learning">¶</a></h1><h2 id="Practice-Session:-Adversarial-Example">Practice Session: Adversarial Examples<a class="anchor-link" href="#Practice-Session:-Adversarial-Example">¶</a></h2><p>In this practice, we will introduce an important attack method: adversarial example, which is executed against the machine learning model (deep learning or neural network). The dataset we will use today is the MNIST dataset of hand-written digitals. Please refer to details of the dataset at <a href="http://yann.lecun.com/exdb/mnist/">http://yann.lecun.com/exdb/mnist/</a>.</p>
<p>As we introduced in this module, adversarial examples are produced by adding some noise on the original image which can lead to a mis-classification from the machine learning model. In today's demonstration, we will train a deep learning model using the same technique as last week, and test its accuracy. We will then choose one image from the training dataset and add some randomised noise on it, and check the prediction result from the deep learning model to see if the model will mis-classify this noisy image.</p>



<p>Firstly, we will import some packages. As you can see below, we'll be using numpy, as well as the <strong>Pytorch</strong> package to construct our neural network for the deep learning model, so ensure you have installed this using the information in your orientation materials.</p>



In [ ]:

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

import numpy as np
import math
import random




<p>Next, we declare some hyper parameters for model training, including <strong>training epochs</strong>, <strong>batch size</strong> and <strong>learning rate</strong>.</p>


In [ ]:

# Hyper Parameters
NUM_EPOCHS = 1
BATCH_SIZE = 100
LEARNING_RATE1 = 0.001




<p>Now, we need to load our <strong>training dataset</strong> and <strong>test dataset</strong>. We're using the MNIST dataset, which is already built in Pytorch's torchvision package. We now load training dataset and test dataset from torchvision.</p>


In [ ]:
# MNIST Dataset
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = [0.5],std = [0.5])
                               ])

# Download training dataset
train_dataset = dsets.MNIST(root = './data/',
                            train = True, 
                            transform = transform,
                            download = True)

# Download testing dataset
test_dataset = dsets.MNIST(root = './data/',
                           train = False, 
                           transform = transform,
                           download = True)

# Data Loader (Input Pipeline)
# The first training dataset for target model
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE, 
                                           shuffle = True)

# The original test dataset
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          shuffle = False)




<p>Let's see the number of images in these datasets. Click run to ouput these numbers.</p> 


In [ ]:

print("The length of training dataset: ", len(train_dataset))
print("The length of test dataset: ", len(test_dataset))



<p>So we can see that we have 60000 training images and 10000 test images for digits from 0 to 9.</p>

<p>Now, let's show the first image of the whole training dataset.</p>


In [ ]:

train_dataset[0]




<p>The training data includes two parts, the <strong>image information</strong> and its <strong>label</strong>. As the result above shows, the first image of the entire training dataset is 5. Let's now print out the image. We will import the matplotlib package and use the <strong>imshow</strong> function, which generates images based on 2-dimensional numpy arrays.</p> You can show any image from the data set by changing the first value following train_dataset. Try it now by entering any number between 0 and 59999, but don't forget to return to the original value 0 before you continue. If you want to show more than one image from the dataset, simply copy the last four lines of code, paste it below the existing code and specifiy the image you would like to display.</p>


In [ ]:

import matplotlib.pyplot as plt
img = train_dataset[0][0].squeeze(0).data
img = img.numpy()
plt.imshow(img)
plt.show()


<p>Now, let's build our model. In the code below, we construct a 4-layer neural network for our deep learning model. This is deep enough for this dataset to reach a very high test accuracy. We use <strong>Cross Entropy Loss</strong> to calculate the gap between the predicted label and real label.</p>

In [ ]:

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
            nn.BatchNorm2d(128, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, dilation = 1, ceil_mode = False),
            nn.ReLU(inplace = True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),
            nn.BatchNorm2d(256, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, dilation = 1, ceil_mode = False),
            nn.ReLU(inplace = True)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1)),             
            nn.BatchNorm2d(512, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, dilation = 1, ceil_mode = False),
            nn.ReLU(inplace = True)
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features = 8192, out_features = 50, bias = True),
            nn.Dropout(p = 0.5),
            nn.Linear(in_features = 50, out_features = 10, bias = True)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Define a model cnn belongs to CNN class
classifierModel = Classifier()
# Loss and Optimizer
criterion1 = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(classifierModel.parameters(), lr = LEARNING_RATE1)




<p>Let's train our model. We train the model for 10 epochs, which means all training data will pass the model 10 times. The training process may cost different lengths of time due to the difference of calculation capability between different devices.</p>


In [ ]:

# Train the Model
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
#         print(labels)
#         print(type(labels))
        
#         labels = dirtyLabel(labels, 7, 1)
#         print(labels)
#         print(type(labels))
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer1.zero_grad()
        outputs = classifierModel(images)
        
        loss = criterion1(outputs, labels)
        loss.backward()
        optimizer1.step()
        
        if (i + 1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch + 1, NUM_EPOCHS, i + 1, len(train_dataset)//BATCH_SIZE, loss.data))




<p>Let's test our model using the test dataset. Click 'run' to output an accuracy report for the ten digits.</p>


In [ ]:

# Test the Model
classifierModel.eval() # Change model to 'eval' mode (BN uses moving mean/var).

correct = 0
total = 0
correct0 = correct1 = correct2 = correct3 = correct4 = correct5 = correct6 = correct7 = correct8 = correct9 = 0
total0 = total1 = total2 = total3 = total4 = total5 = total6 = total7 = total8 = total9 = 0

names = locals()

for images, labels in test_loader:            
    images = Variable(images)
    outputs = classifierModel(images)
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
    labels = labels.tolist()
    for l in range(0, 10):
        if labels[0] == l:
            names["total" + str(l)] = names["total" + str(l)] + 1
            predicted = predicted.tolist()
            if predicted[0] == labels[0]:
                names["correct" + str(l)] = names["correct" + str(l)] + 1

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * float(correct) / total))

for num in range(0, 10):
    print("Test Accuracy of the model on the number", num, "is: %d %%" % (100 * float(names["correct" + str(num)]) / names["total" + str(num)]))




<p>The result above shows that our model can reach over 99% accuracy. Now, let's test the predicted label for the image 5 we printed out.</p>


In [ ]:

classifierModel.eval()
images = torch.tensor(img, dtype = torch.float32)
images = images.unsqueeze(0)
images = images.unsqueeze(0)
print(images.shape)
print(images.type())
images = Variable(images)
outputs = classifierModel(images)
_, predicted = torch.max(outputs.data, 1)
print(predicted)



## Making an adversarial image

<p>Let's make this image "adversarial" by adding some perturbed noise on this image. The image will look similar to the original image, so to us it will still look like the number 5. However, our trained model will mis-classify it as another digit.</p>
<p>Firstly, we generate a noisy tensor with the same shape as our original image, 32 x 32 pixels. We use the <strong>random</strong> function of the numpy package to generate a random noise matrix. We need to carefully scale the noise and guarantee the noise won't change the image visually too much. The <strong>random.random()</strong> function will generate a random number in the range (0,1).</p>


In [ ]:

noiseMatrix = np.random.random((32,32))
print(noiseMatrix)




<p>We add our generated noise on the original image, and print out the noisy image.</p>


In [ ]:

img1 = img + noiseMatrix 
plt.imshow(img1)
plt.show()




<p>From the new perturbed image we can find that, although the image is noisy comparing with the original image, we can still visually recognise the number from the perturbed image is 5. Now, we need to convert the numpy matrix of the image to a tensor, for the deep learning model to do the prediction.</p>


In [ ]:

images = torch.tensor(img1, dtype = torch.float32)
images = images.unsqueeze(0)
images = images.unsqueeze(0)
print(images.shape)
print(images.type())




<p>Now, let's bring this perturbed image to our trained model, and see the prediction result.</p>


In [ ]:

classifierModel.eval()
images = Variable(images)
outputs = classifierModel(images)
_, predicted = torch.max(outputs.data, 1)
print(predicted)




<p>The prediction result from the deep learning model now is changed to 8. This is our adversarial example against a deep learning model.</p>
<p>Finally, we scale the perturbation and see the comparison between the original image and perturbed images, and the prediction results from the deep learning model.</p>
<p>We scale down our noise to 1/2, 1/3, 1/4 and 1/5.</p>


In [ ]:

for i in range(2, 6):
    
    img1 = img + noiseMatrix / i
    print("----- Noise is scaled down to 1/%d ----" % i)
    plt.figure(1)
    plt.subplot(221)
    plt.imshow(img)
    plt.subplot(222)
    plt.imshow(img1)
    plt.show()
    
    images = torch.tensor(img1, dtype = torch.float32)
    images = images.unsqueeze(0)
    images = images.unsqueeze(0)
    
    classifierModel.eval()
    images = Variable(images)
    outputs = classifierModel(images)
    _, predicted = torch.max(outputs.data, 1)
    
    print("-- The prediction result now is: %d --" % predicted.data)




<p>From the results above, you can see that the image looks clearer as the noise gets smaller, but the prediction result tends to be become more accurate. Therefore, it's important for an attacker to add the smallest possible amount of noise on the original image to guarantee visual clarity that will still mislead the machine learning model to mis-classify it.</p>
